In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2 
import pandas as pd
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.optimizers import RMSprop
import imghdr
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization

In [ ]:
data_dir = '/Users/zahidlaguna/Desktop/TrainingData'
img_ext = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
for image_name in os.listdir(data_dir):
    if os.path.isfile(os.path.join(data_dir, image_name)) and imghdr.what(os.path.join(data_dir, image_name)) not in img_ext:
        print(image_name)
        os.remove(os.path.join(data_dir, image_name))

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
for image_name in os.listdir(data_dir):
    if os.path.isdir(os.path.join(data_dir, image_name)):
        continue
    if os.path.isfile(os.path.join(data_dir, image_name)) and imghdr.what(os.path.join(data_dir, image_name)) not in img_ext:
        print(f"Skipping {image_name} - unsupported file format")
        continue
    print(f"Processing {image_name}")
    img = image.load_img(os.path.join(data_dir, image_name), target_size=(150, 150))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    batch = np.vstack([x])


In [ ]:
#plot the image data
plt.figure(figsize=(10, 10))
for images, labels in data.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        try:
            plt.imshow(images[i].numpy().astype("uint8"))
        except Exception as e:
            print(f"Skipping image - {e}")
            continue
        plt.title(int(labels[i]))
        plt.axis("off")
plt.show()